# Point-to-Mesh Similarity Analysis

This notebook demonstrates how to:
1. Load two meshes
2. Specify a particular point on the source mesh
3. Color-map the target mesh based on similarity to that point

The analysis uses diffusion features + DINO features to compute semantic similarity between mesh vertices.

## 1. Import Dependencies

In [1]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map
import importlib
import meshplot as mp
from point_to_mesh_similarity import run_point_similarity_analysis, point_similarity_colormap, visualize_point_similarity, run_multi_point_correspondence_analysis
import numpy as np

%load_ext autoreload
%autoreload 2

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/home/zixiliu/miniforge3/envs/diff3f_copy2/lib/python3.10/site-packages/accelerate/utils/torch_xla.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Using device: cuda


## 2. Initialize Models

This step loads the diffusion pipeline and DINO model for feature extraction.

In [2]:
print("Initializing diffusion pipeline...")
pipe = init_pipe(device)

print("Initializing DINO model...")
dino_model = init_dino(device)

print("Models initialized successfully!")

Initializing diffusion pipeline...


/home/zixiliu/miniforge3/envs/diff3f_copy2/lib/python3.10/site-packages/diffusers/configuration_utils.py:239: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Initializing DINO model...


Using cache found in /home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


Models initialized successfully!


## 3. Load Meshes

Load your source and target meshes. You can modify the paths below to use your own meshes.

In [3]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_bottle_decomp.obj"

source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)


print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 9041 vertices, 17850 faces


## 4. Preview Meshes

Let's visualize the meshes before analysis to get familiar with their structure.

In [4]:
if 0:
    # Preview the meshes side by side
    d = mp.subplot(source_mesh.vert[:,0:3], source_mesh.face, s=[2, 2, 0])
    mp.subplot(target_mesh.vert[:,0:3], target_mesh.face, s=[2, 2, 1], data=d)
    print("Left: Source mesh | Right: Target mesh")

## 5. Configure Analysis Parameters

Set up the parameters for the similarity analysis.

In [5]:
# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")


p1 = [ 0.00886742, -0.04124315, -0.01754964]
p2 = [-0.01223545,  0.0363251 ,  0.00719686]
p3 = [-0.01694481,  0.03424783, -0.01735426]
p4 = [-0.01596961,  0.03464369, -0.02075735]
source_points_3d = np.array([p1, p2, p3, p4])


Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10


### To clean up meshes

In [6]:
if 0:
    from utils import clean_obj_file_format_only
    source_fixed = clean_obj_file_format_only("meshes/oakink_beaker_decomp2.obj")
    target_fixed = clean_obj_file_format_only("meshes/oakink_mug_decomp2.obj")
    
    # Now load the new target mesh with your MeshContainer
    source_mesh_path = "meshes/oakink_beaker_decomp2_fixed.obj"
    target_mesh_path = "meshes/oakink_mug_decomp2_fixed.obj"

### To clean up GPU cache

In [7]:
if 0:
    import torch
    
    # Clear cache of unused memory
    torch.cuda.empty_cache()
    
    # Optionally, collect unused memory from Python garbage collector
    import gc
    gc.collect()
    torch.cuda.empty_cache()

# Explore Re-mesh Open3D

In [ ]:
if 0:
    mesh = o3d.io.read_triangle_mesh(target_mesh_path)
    
    # Create a point cloud with a uniform density using Poisson disk sampling
    # The number_of_points parameter controls the point density.
    # A higher number will result in a finer, more detailed final mesh.
    pcd = mesh.sample_points_poisson_disk(number_of_points=10000)
    
    # Optional: Visualize the point cloud to inspect the distribution
    o3d.visualization.draw_geometries([pcd])
    
    # The alpha parameter controls the size of the spheres used to create the mesh.
    # A smaller alpha will produce a tighter mesh that follows the point cloud more closely,
    # while a larger alpha will create a coarser mesh.
    alpha = 0.005
    new_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    
    # Compute vertex normals for correct lighting and shading
    new_mesh.compute_vertex_normals()
    
    # Visualize the new mesh
    o3d.visualization.draw_geometries([new_mesh], mesh_show_back_face=True)
    
    # Optional: Save the new mesh
    # o3d.io.write_triangle_mesh("uniform_remeshed.ply", new_mesh)

    # Convert mesh to LineSet (wireframe representation)
    wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(new_mesh)
    wireframe.paint_uniform_color([0, 0, 0])  # black wireframe
    
    # Show wireframe
    o3d.visualization.draw_geometries([wireframe])
    


## 6. Run Similarity Analysis

This is the main computation step. It will:
1. Extract features for both meshes using multiple camera views
2. Compute similarity between the specified source point and all target vertices
3. Generate the visualization

⚠️ **Note**: This step can take several minutes depending on mesh complexity and number of views.

In [ ]:
remesh = True

if 1:
    source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
    target_mesh_path = "meshes/oakink_bottle_decomp2.obj"
    num_views = 10  # Number of views for rendering (reduced for faster computation)

    if remesh: 
        from utils import remesh_mesh_pair
        source_mesh_path, target_mesh_path = remesh_mesh_pair(
            source_mesh_path,
            target_mesh_path,
            num_points=10000,
            alpha=0.005
        )

    
    source_mesh = MeshContainer().load_from_file(source_mesh_path)
    target_mesh = MeshContainer().load_from_file(target_mesh_path)

    
    print("Source mesh vert:", source_mesh.vert.shape)
    print("Target mesh vert:", target_mesh.vert.shape)
    
    # Run the complete analysis pipeline
    similarity_colors, raw_similarities, source_point_idx, closest_distance = run_point_similarity_analysis(
        source_mesh=source_mesh,
        target_mesh=target_mesh,
        source_point_3d=p1,
        device=device,
        pipe=pipe,
        dino_model=dino_model,
        prompt=prompt,
        num_views=num_views
    )
    
    print("\n✅ Analysis complete!")

[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
✅ Re-mesh complete: meshes/oakink_beaker_decomp2_remesh.obj, meshes/oakink_bottle_decomp2_remesh.obj
Source mesh vert: (9291, 3)
Target mesh vert: (9083, 3)
Input 3D point: [ 0.00886742 -0.04124315 -0.01754964]
Closest vertex: 2297 (distance: 0.0042)
Closest vertex coordinates: [ 0.00722481 -0.0373988  -0.0171679 ]


/home/zixiliu/miniforge3/envs/diff3f_copy2/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 70%|█████████████████████▋         | 7/10 [00:16<00:06,  2.27s/it]

# Multi point analysis

In [ ]:
## NOTE: no difference between oakink_beaker_decomp2.obj and oakink_beaker_decomp.obj
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_bottle_decomp.obj"

source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
num_views = 20  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 9041 vertices, 17850 faces
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [ 0.00886742 -0.04124315 -0.01754964] -> Vertex 5589 (distance: 0.0100)
  Point 1: [-0.01223545  0.0363251   0.00719686] -> Vertex 9138 (distance: 0.0116)
  Point 2: [-0.01694481  0.03424783 -0.01735426] -> Vertex 2681 (distance: 0.0060)
  Point 3: [-0.01596961  0.03464369 -0.02075735] -> Vertex 2717 (distance: 0.0053)
Computing features for source mesh...


100%|██████████████████████████████| 20/20 [00:53<00:00,  2.66s/it]


Number of missing features:  521 . Copied features from nearest vertices.
Time taken in mins:  0.9516005992889405
Computing features for target mesh...


 15%|████▋                          | 3/20 [00:07<00:41,  2.43s/it]

# A few more examples

In [ ]:
num_views = 10


source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_mug_decomp2.obj"
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
num_views = 10  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_mug_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

num_views = 4  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_mug_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_wineglass_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_wineglass_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 6  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_bowl_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 8  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_mug2_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_vase_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

In [ ]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_jar_decomp.obj"

# source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)
print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

## 7. Analyze Results

Let's examine the similarity scores and find the most similar regions.

In [ ]:
# Find the most similar points on target mesh
top_k = 10
most_similar_indices = np.argsort(raw_similarities)[-top_k:][::-1]
least_similar_indices = np.argsort(raw_similarities)[:top_k]

print(f"📊 Similarity Statistics:")
print(f"- Min similarity: {raw_similarities.min():.4f}")
print(f"- Max similarity: {raw_similarities.max():.4f}")
print(f"- Mean similarity: {raw_similarities.mean():.4f}")
print(f"- Std similarity: {raw_similarities.std():.4f}")

print(f"\n🔥 Top {top_k} most similar vertices on target mesh:")
for i, idx in enumerate(most_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

print(f"\n❄️  Bottom {top_k} least similar vertices on target mesh:")
for i, idx in enumerate(least_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

## 8. Visualize Similarity Distribution

Plot the distribution of similarity scores to understand the data better.

In [ ]:
import matplotlib.pyplot as plt 

# Plot similarity distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(raw_similarities, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(raw_similarities.mean(), color='red', linestyle='--', label=f'Mean: {raw_similarities.mean():.3f}')
plt.axvline(raw_similarities.max(), color='green', linestyle='--', label=f'Max: {raw_similarities.max():.3f}')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Similarity Scores')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(sorted(raw_similarities, reverse=True), linewidth=2)
plt.xlabel('Vertex Rank (sorted by similarity)')
plt.ylabel('Similarity Score')
plt.title('Similarity Scores (Ranked)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Interactive Visualization

The main result visualization showing:
- **Left**: Source mesh (gray) with the selected point highlighted in red
- **Right**: Target mesh colored by similarity (warmer colors = more similar)

You can interact with the 3D visualization to rotate and zoom the meshes.

In [ ]:
print("🎨 Interactive Visualization:")
print("- Source mesh (left): Gray with red point showing the reference vertex")
print("- Target mesh (right): Colored by similarity (warmer colors = more similar)")
print("- Use mouse to rotate and zoom the 3D view")

# The visualization was already created by run_point_similarity_analysis
# But let's create it again with custom colormap options
visualize_point_similarity(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_point_idx=source_point_idx,
    similarity_colors=similarity_colors,
)

## 10. Experiment with Different Points

Try analyzing different points on the source mesh to see how similarity patterns change.

In [ ]:
# Experiment with different source points
# You can change these indices and re-run this cell
experiment_points = [100, 500, 1500, 2000]  # Different vertex indices to try

for point_idx in experiment_points:
    if point_idx < len(source_mesh.vert):
        print(f"\n🔬 Experimenting with source vertex {point_idx}...")

        # Quick analysis with fewer views for faster computation
        exp_colors, exp_similarities = point_similarity_colormap(
            device=device,
            pipe=pipe,
            dino_model=dino_model,
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            prompt=prompt,
            num_views=10  # Fewer views for quick experiment
        )

        # Show quick stats
        top_match = np.argmax(exp_similarities)
        print(f"  Best match: vertex {top_match} (similarity: {exp_similarities[top_match]:.4f})")

        # Visualize
        visualize_point_similarity(
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            similarity_colors=exp_colors,
            colormap='viridis'
        )
    else:
        print(f"Skipping vertex {point_idx} (mesh only has {len(source_mesh.vert)} vertices)")

## 11. Save Results (Optional)

Save the similarity data for later analysis.

In [ ]:
# Save results to files
output_prefix = f"similarity_results_vertex_{source_point_idx}"

# Save similarity scores
np.save(f"{output_prefix}_similarities.npy", raw_similarities)
np.save(f"{output_prefix}_colors.npy", similarity_colors)

# Save analysis summary
summary = {
    'source_mesh_path': source_mesh_path,
    'target_mesh_path': target_mesh_path,
    'source_point_idx': source_point_idx,
    'prompt': prompt,
    'num_views': num_views,
    'min_similarity': float(raw_similarities.min()),
    'max_similarity': float(raw_similarities.max()),
    'mean_similarity': float(raw_similarities.mean()),
    'std_similarity': float(raw_similarities.std()),
    'top_matches': [int(idx) for idx in most_similar_indices[:5]]
}

import json
with open(f"{output_prefix}_summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✅ Results saved:")
print(f"  - Similarities: {output_prefix}_similarities.npy")
print(f"  - Colors: {output_prefix}_colors.npy")
print(f"  - Summary: {output_prefix}_summary.json")

## Summary

This notebook demonstrated how to:

1. ✅ Load and visualize 3D meshes
2. ✅ Extract semantic features using diffusion + DINO models
3. ✅ Compute similarity between a specific source point and all target vertices
4. ✅ Visualize results with interactive 3D plots
5. ✅ Analyze similarity distributions and find best matches
6. ✅ Experiment with different source points
7. ✅ Save results for future analysis

### Next Steps

- Try different mesh pairs
- Experiment with different prompts
- Adjust the number of views for quality vs. speed trade-offs
- Use the saved similarity data for further analysis or applications